## Importer les librairies

In [2]:
#! pip install numpy 
#! pip install pandas 
#! pip install matplotlib 
#! pip install plotly 
#! pip install nltk 
#! pip install textblob 
#! pip install wordcloud 
#! pip install scikit-learn
#! pip install sklearn 
#! pip install openpyxl
#! pip install nbformat==4.2.0
#! python -m textblob.download_corpora


In [3]:
# Basic Libraries 📚
# --------------------------------------
import numpy as np
import pandas as pd
import re

# Plot library 📊
# --------------------------------------
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio


# NLP
# --------------------------------------
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import Word, TextBlob
from wordcloud import WordCloud  # visualization of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# Metrics 📐
# --------------------------------------
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder


# Machine Learning Models 🤖
# --------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# Customize to Remove Warnings and Better Observation 🔧
# --------------------------------------------------------
from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Problèmatique

Augmenter ses ventes en analysant les commentaires reçus sur des produits, et améliorer leurs caractéristiques en fonction des plaintes reçues. Conformément à cet objectif, les commentaires seront étiquetés en effectuant une analyse des sentiments et un modèle de classification sera créé avec les données étiquetées.

### Les données

L'ensemble de données comprend les commentaires faits pour un certain groupe de produits, le titre du commentaire, le nombre d'étoiles et les variables qui indiquent combien de personnes ont trouvé le commentaire utile.

### Import du dataset :

In [4]:
df = pd.read_excel("amazon.xlsx")
df.head(15)

,Star,HelpFul,Title,Review
0,5,0,looks great,Happy with it
1,5,0,Pattern did not align between the two panels.,Good quality material however the panels are m...
2,5,0,Imagery is stretched. Still fun.,Product was fun for bedroom windows.<br />Imag...
3,5,0,Que se ven elegantes muy finas,Lo unico que me gustaria es que sean un poco ...
4,5,0,Wow great purchase,Great bang for the buck I can't believe the qu...
5,5,0,Not for us,Looks different then photo
6,5,0,Perfect for spa room,Was exactly what i was looking for. Heavy mate...
7,5,0,Actually better than expected,"Looking at the picture, I thought these curtai..."
8,4,0,Not what expected,Much whiter than I thought it would be and was...
9,5,0,Pretty as a Picture,These are curtains just as the picture shows. ...


#### Suppression des nulls

In [5]:
print("Il y a",df['Review'].isnull().sum(),"avis sans commentaire(s)")

Il y a 18 avis sans commentaire(s)


In [6]:
df.dropna(subset=['Review'], inplace=True)
df.isnull().sum()

Star        0
HelpFul     0
Title      43
Review      0
dtype: int64

## Pré Traitement du texte

### Minuscule

In [7]:
# to put all the characters in a standard
print(df['Review'].head())
df['Review'] = df['Review'].str.lower()  # make all lowercase
df['Review'].head()

0                                        Happy with it
1    Good quality material however the panels are m...
2    Product was fun for bedroom windows.<br />Imag...
3     Lo unico que me gustaria es que sean un poco ...
4    Great bang for the buck I can't believe the qu...
Name: Review, dtype: object


0                                        happy with it
1    good quality material however the panels are m...
2    product was fun for bedroom windows.<br />imag...
3     lo unico que me gustaria es que sean un poco ...
4    great bang for the buck i can't believe the qu...
Name: Review, dtype: object

### Ponctuation

Tout comme les majuscules et les minuscules, les signes de ponctuation ne contiennent pas de valeur de mesure. Par exemple, si nous pensons que nous faisons un exemple de classification, il y a 2 avis, l'un d'eux a beaucoup plus de points/virgules. Dans cette situation, nous n'attendons AUCUN modèle des ponctuations. Bien qu'il s'agisse de l'approche courante, elle peut varier en fonction du problème.


In [16]:
# check sentences and replace punctions with spaces
print(df['Review'].head())

df['Review'] =  df['Review'].apply(lambda x: re.sub(r'[^\w\s]',' ', x))


print(df['Review'].head())


0                                                happy
1       good quality material however panel mi matched
2    product fun bedroom window imagery bit stretch...
3    lo unico que gustaria e que sean un poco ma la...
4    great bang buck believe quality material vibra...
Name: Review, dtype: object
0                                                happy
1       good quality material however panel mi matched
2    product fun bedroom window imagery bit stretch...
3    lo unico que gustaria e que sean un poco ma la...
4    great bang buck believe quality material vibra...
Name: Review, dtype: object


### Chiffres

RAPPEL :
Comme ci-dessus, notre problématique peut avoir une discrimination sur les nombres. Par exemple, inclure beaucoup ou moins de nombres peut être crucial pour le problème. Dans ce type de problème, la suppression des nombres n'est pas une méthode à utiliser. Dans notre exemple, nous supprimons les nombres car ils ne sont pas significatifs.

L'approche générale consiste à supprimer toutes les structures problématiques en dehors du texte. Par exemple, lorsqu'il y a des données de médias sociaux, nous pouvons choisir de supprimer les émojis, les liens de pages, etc.

Ici, Nous détectons les chiffres via une expression régulière. \d nous aide à capturer les chiffres.


In [9]:
print(df['Review'].head())

df['Review'] = df['Review'].str.replace('\d', '')  

print(df['Review'].head())


0                                        happy with it
1    good quality material however the panels are m...
2    product was fun for bedroom windows.<br />imag...
3     lo unico que me gustaria es que sean un poco ...
4    great bang for the buck i can't believe the qu...
Name: Review, dtype: object
0                                        happy with it
1    good quality material however the panels are m...
2    product was fun for bedroom windows.<br />imag...
3     lo unico que me gustaria es que sean un poco ...
4    great bang for the buck i can't believe the qu...
Name: Review, dtype: object


### Balises HTML

In [10]:
CLEANR = re.compile('<.*?>') 

print(df['Review'].head())

df['Review'] = df['Review'].apply(lambda x: re.sub(CLEANR, ' ', x))

print(df['Review'].head())

0                                        happy with it
1    good quality material however the panels are m...
2    product was fun for bedroom windows.<br />imag...
3     lo unico que me gustaria es que sean un poco ...
4    great bang for the buck i can't believe the qu...
Name: Review, dtype: object
0                                        happy with it
1    good quality material however the panels are m...
2    product was fun for bedroom windows. imagery a...
3     lo unico que me gustaria es que sean un poco ...
4    great bang for the buck i can't believe the qu...
Name: Review, dtype: object


### Caractère spéciaux

In [11]:
clean = re.compile(r'[^a-zA-Z0-9]')
df['Review'] = df['Review'].apply(lambda x: re.sub(clean, ' ', x))

### Stopword

Les mots couramment utilisés dans la langue n'ont pas de mesure. Des expressions telles que diverses conjonctions telles que est, pour, ceci ou divers pronoms n'ont pas de signification métrique. Par conséquent, nous supprimerons ces expressions.

Nous devons télécharger « stopwords » de la bibliothèque nltk. Il a été préparé pour l'anglais, il existe également des études pour d'autres langues. Par conséquent, vous pouvez utiliser la langue que vous souhaitez comme argument.

In [12]:
nltk.download('stopwords')
sw = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p115302\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
df['Review'].head(5)

0                                                happy
1     good quality material however panels mis matched
2    product fun bedroom windows imagery bit stretc...
3    lo unico que gustaria es que sean un poco mas ...
4    great bang buck believe quality material vibra...
Name: Review, dtype: object

### Lemmatization


In [14]:
nltk.download('wordnet')
nltk.download('omw-1.4')

df['Review'] = df['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df['Review'].head(5)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p115302\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\p115302\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0                                                happy
1       good quality material however panel mi matched
2    product fun bedroom window imagery bit stretch...
3    lo unico que gustaria e que sean un poco ma la...
4    great bang buck believe quality material vibra...
Name: Review, dtype: object